<a href="https://colab.research.google.com/github/BrooklynZhang/Personal/blob/master/AntColonyOpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Define the HyperParameters

numNodes = 10       # Number of Nodes
numAnts = 100       # Number of Ants
numpheAnts = 50     # Number of Ants which could spread pheormones
decay = 0.95        # Pheromone decays
alpha = 4           # Weight of pheromone
beta = 1            # Weight of distance
iterations = 100

In [0]:
#Auto generate a graph which indicates the distance between each 2 nodes.
#The distance between each 2 nodes are random number from 1 to 100
#graph[1][3] = 75 indicates the distance between 1 and 3 is 75

import random

def init_graph():
  numNodesList = [[0 for i in range(numNodes)] for j in range(numNodes)]
  for i in range(numNodes):
    for j in range(numNodes):
      if i == j:
        numNodesList[i][j] = np.inf
      else:
        len = random.randint(1,100)
        numNodesList[i][j] =  numNodesList[j][i] = len
  return numNodesList

In [0]:
import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):
  def __init__(self, graph):
    self.graph = graph                # The distance graph of the nodes
    self.pheromone = np.ones(self.graph.shape) / numNodes


  def start(self):
    current_shortest_path = None
    result = [np.inf, np.inf]
    for i in range(iterations):
      paths = self.generate_all_paths()                         #each ants generate a path based on the current pheronomes and distance                  
      self.spread_pheronome(paths)                              #for each path, spread the pheronmos
      current_shortest_path = min(paths, key=lambda x: x[1])    #find the least distance path
      print (current_shortest_path)                             #print out
      if current_shortest_path[1] < result[1]:                  #compare based on the distance
        result = current_shortest_path                          
      self.pheromone * decay                                    #decays of pheronmeos
    return result


  def spread_pheronome(self, path):
    sorted_paths = sorted(path, key=lambda x: x[1]) #sorted based on the distance
    for p, d in sorted_paths[:numpheAnts]:
      for step in p:
        #print(step)
        self.pheromone[step] += 1.0 / self.graph[step]  # if distance of 1,2 is 4 and 2,5 is 5, then the pheromone added to the path 1,2 is 1/4, add to 2,5 is 1/5

  def generate_all_paths(self):
    result = []
    for i in range(numAnts):          # for each ants find a path
      path = self.gen_path(0)
      result.append((path, self.generate_path_distance(path)))
    return result

  def generate_path_distance(self, path):
    result = 0
    for i in path:
      result += self.graph[i]
    return result

  def gen_path(self,start_node):
    path = []
    visited_node = set()
    visited_node.add(start_node)
    prev_node = start_node
    for i in range(numNodes - 1):
      next_node = self.select_path(self.pheromone[prev_node], self.graph[prev_node], visited_node) # select a path based on the distance/phenomones
      path.append((prev_node, next_node))
      prev_node = next_node
      visited_node.add(next_node)
    path.append((prev_node, start_node))  
    return path

  def select_path(self, pheromone, distances, visited):
    current_pheromone = np.copy(pheromone)
    current_pheromone[list(visited)] = 0
    scores = current_pheromone ** alpha * (( 1.0 / distances) ** beta)    # Score of one node to another node = (pheromone ** alpha) * ((1 / distance) ** beta)
                                                                          # alpha is the weight on pheromone, beta is distance's
                                                                    # there is no circle in the shorest path because it is not allowe
    norm_score = scores / scores.sum()                              # score / total scores of avaliable nodes is the probility to select that node
    next_step = np_choice(numNodes, 1, p=norm_score)[0]             # choose 1 nodes from numNodes based on the probility list
    return next_step
    




In [0]:
# Main Function

if __name__ == "__main__":
  print("Ant Colony Opt Alg Start")
  Graph = np.array(init_graph())
  Ant_Colony = AntColony(Graph)
  print("Initial Graph is \n", Graph)

  shortest_path = Ant_Colony.start()
  print ("shorted_path: {}".format(shortest_path))

Ant Colony Opt Alg Start
Initial Graph is 
 [[inf 28. 91. 66. 20. 56. 97. 26. 69. 85.]
 [28. inf 16. 41. 67. 63. 31. 48. 29. 80.]
 [91. 16. inf 28.  7. 53. 31. 38. 20. 74.]
 [66. 41. 28. inf 19. 37. 17. 44. 88. 17.]
 [20. 67.  7. 19. inf  7. 81. 82. 21. 27.]
 [56. 63. 53. 37.  7. inf 66. 71. 78. 26.]
 [97. 31. 31. 17. 81. 66. inf 98. 18. 56.]
 [26. 48. 38. 44. 82. 71. 98. inf 37.  5.]
 [69. 29. 20. 88. 21. 78. 18. 37. inf 11.]
 [85. 80. 74. 17. 27. 26. 56.  5. 11. inf]]
([(0, 4), (4, 5), (5, 9), (9, 8), (8, 6), (6, 1), (1, 2), (2, 3), (3, 7), (7, 0)], 227.0)
([(0, 7), (7, 9), (9, 8), (8, 6), (6, 3), (3, 5), (5, 4), (4, 2), (2, 1), (1, 0)], 172.0)
([(0, 4), (4, 5), (5, 3), (3, 6), (6, 8), (8, 9), (9, 7), (7, 2), (2, 1), (1, 0)], 197.0)
([(0, 4), (4, 5), (5, 9), (9, 7), (7, 8), (8, 6), (6, 3), (3, 2), (2, 1), (1, 0)], 202.0)
([(0, 4), (4, 5), (5, 9), (9, 7), (7, 8), (8, 6), (6, 3), (3, 2), (2, 1), (1, 0)], 202.0)
([(0, 4), (4, 5), (5, 9), (9, 7), (7, 8), (8, 6), (6, 3), (3, 2), (2, 1), (